In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from preprocessing_data.utils import format_Dataframes, preprocessing_dataframe

In [2]:
df = format_Dataframes(data_path="D:\-DSP391m-Forecasting-Financial-Time-Series-With-Transformer\craw_data\FPT_stock.xlsx", type_file="xlsx")
df.head(3)

In [3]:
df.tail(3)

In [4]:
df = preprocessing_dataframe(df)

In [5]:
df

In [17]:
for i in df["Đóng cửa"]:
    if i == 0:
        print(i)

In [7]:
def plot_price_changes(df: pd.DataFrame) -> None:
    fig = plt.figure(figsize=(10, 6))
    df = df.sort_values(by='Ngày')
    sns.lineplot(x='Ngày', y='Đóng cửa', data=df, label='Giá đóng cửa')
    sns.lineplot(x='Ngày', y='Mở cửa', data=df, label='Giá mở cửa')
    plt.title('Sự thay đổi của giá đóng và giá mở theo thời gian')
    plt.xlabel('Ngày')
    plt.ylabel('Giá')
    plt.legend()
    plt.xticks(rotation=45) 
    plt.tight_layout()
    plt.show()

In [8]:
def plot_trading_volume(df: pd.DataFrame) -> None:
    fig = plt.figure(figsize=(10, 6))
    df = df.sort_values(by='Ngày')
    sns.lineplot(x='Ngày', y='Khối lượng (Khớp lệnh)', data=df, label='Khối lượng (Khớp lệnh)')
    sns.lineplot(x='Ngày', y='Khối lượng (Thỏa thuận)', data=df, label='Khối lượng (Thỏa thuận)')
    plt.title('Sự thay đổi của khối lượng giao dịch khớp lệnh và thỏa thuận theo thời gian')
    plt.xlabel('Ngày')
    plt.ylabel('Khối lượng')
    plt.legend()
    plt.xticks(rotation=45) 
    plt.tight_layout()
    plt.show()

In [9]:
def plot_price_range(df: pd.DataFrame) -> None:
    fig = plt.figure(figsize=(10, 6))
    df = df.sort_values(by='Ngày')
    df['Chênh lệch giá'] = df['Cao nhất'] - df['Thấp nhất']
    sns.lineplot(x='Ngày', y='Chênh lệch giá', data=df)
    plt.title('Sự chênh lệch giữa giá cao nhất và thấp nhất theo thời gian')
    plt.xlabel('Ngày')
    plt.ylabel('Chênh lệch giá')
    plt.xticks(rotation=45) 
    plt.tight_layout() 
    plt.show()


In [10]:
def plot_correlation_heatmap(df: pd.DataFrame) -> None:
    numeric_cols = df.select_dtypes(include=['float64']).columns
    corr_matrix = df[numeric_cols].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Biểu đồ heatmap của mối tương quan giữa các biến')
    plt.show()

In [11]:
# plot_price_changes(df)

In [12]:
# plot_trading_volume(df)

In [13]:
# plot_price_range(df)

In [14]:
# plot_correlation_heatmap(df)